## Categorizing into simple, medium and hard categories

In [1]:
import yaml

# Load the already sampled 100 entries
with open("sampled_100_entries.yml", "r") as f:
    sampled_data = yaml.safe_load(f)

# Prepare YAML block scalar formatting
class LiteralStr(str): pass

def str_presenter(dumper, data):
    return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')

yaml.add_representer(LiteralStr, str_presenter)

# Create buckets
simple, medium, complex = [], [], []

# Classify by code line count
for entry in sampled_data:
    code = entry.get("code", "")
    formatted_code = code.replace("\\n", "\n").replace("\\\\", "\\")
    entry["code"] = LiteralStr(formatted_code)

    line_count = formatted_code.count("\n") + 1
    if line_count <= 9:
        simple.append(entry)
    elif line_count <= 13:
        medium.append(entry)
    else:
        complex.append(entry)

# Save categorized datasets
with open("simple.yml", "w") as f:
    yaml.dump(simple, f, sort_keys=False)

with open("medium.yml", "w") as f:
    yaml.dump(medium, f, sort_keys=False)

with open("complex.yml", "w") as f:
    yaml.dump(complex, f, sort_keys=False)

print("Categorized into simple.yml, medium.yml, and complex.yml.")

FileNotFoundError: [Errno 2] No such file or directory: 'sampled_100_entries.yml'

### lengths of files

In [ ]:
import yaml

# Filenames
files = {
    "Simple": "simple.yml",
    "Medium": "medium.yml",
    "Complex": "complex.yml"
}

# Count entries in each file
for label, filename in files.items():
    with open(filename, "r") as f:
        data = yaml.safe_load(f)
        print(f"{label} complexity: {len(data)} code blocks")

Simple complexity: 35 code blocks
Medium complexity: 35 code blocks
Complex complexity: 30 code blocks


In [ ]:
import yaml

# Filenames
files = {
    "Simple": "simple@1/simple@1_variations.yml",
    "Medium": "medium@1/medium@1_variations.yml",
    "Complex": "complex@1/complex@1_variations.yml"
}

# Count entries in each file
for label, filename in files.items():
    with open(filename, "r") as f:
        data = yaml.safe_load(f)
        print(f"{label} codes: {len(data)} code blocks")

Simple codes: 35 code blocks
Medium codes: 35 code blocks
Complex codes: 30 code blocks


In [ ]:
import yaml

# Filenames
files = {
    "Simple": "simple@1_variations_gpt.yml",
    "Medium": "medium@1_variations_gpt.yml",
    "Complex": "complex@1_variations_gpt.yml"
}

# Count entries in each file
for label, filename in files.items():
    with open(filename, "r") as f:
        data = yaml.safe_load(f)
        print(f"{label} codes: {len(data)} code blocks")

Simple codes: 35 code blocks
Medium codes: 35 code blocks
Complex codes: 30 code blocks


### Validate a single code

In [ ]:
import yaml
import subprocess
import tempfile
from pathlib import Path

# === CONFIGURATION ===
yaml_file = "complex_variations_gpt.yml"  # Change to your file path

# === LOAD YAML FILE ===
try:
    with open(yaml_file, "r") as f:
        codes = yaml.safe_load(f)
except FileNotFoundError:
    print(f"❌ YAML file not found: {yaml_file}")
    exit(1)

if not isinstance(codes, list) or not all("code" in item for item in codes):
    print("❌ Invalid YAML format. Expected a list of items with 'code' fields.")
    exit(1)

# === PROMPT FOR INDEX ===
max_index = len(codes) - 1
print(f"📄 Loaded {len(codes)} code snippets from {yaml_file}")

try:
    index = int(input(f"🔢 Enter index to validate (0–{max_index}): "))
except ValueError:
    print("❌ Invalid input. Please enter an integer.")
    exit(1)

if not (0 <= index <= max_index):
    print(f"❌ Invalid index. Must be between 0 and {max_index}.")
    exit(1)

code = codes[index]["code"].strip()

# === WRITE TO TEMP FILE ===
with tempfile.NamedTemporaryFile(delete=False, suffix=".sv", mode="w") as tmp_file:
    tmp_file.write(code)
    tmp_file_path = Path(tmp_file.name)

# === RUN VERILATOR ===
try:
    print(f"\n🎯 Validating index {index} from {yaml_file}")
    result = subprocess.run(
        ["verilator", "--lint-only", "--timing", str(tmp_file_path)],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        timeout=10
    )
    passed = result.returncode == 0
    print("✅ PASS" if passed else "❌ FAIL")

    if result.stdout:
        print("\nSTDOUT:")
        print(result.stdout.decode().strip())
    if result.stderr:
        print("\nSTDERR:")
        print(result.stderr.decode().strip())

except subprocess.TimeoutExpired:
    print("❌ FAIL — Timeout while linting")
except FileNotFoundError:
    print("❌ FAIL — Verilator not found. Is it installed and in PATH?")
finally:
    tmp_file_path.unlink(missing_ok=True)  # Clean up temp file

📄 Loaded 30 code snippets from complex_variations_gpt.yml

🎯 Validating index 29 from complex_variations_gpt.yml
✅ PASS

STDOUT:
- V e r i l a t i o n   R e p o r t: Verilator 5.036 2025-04-27 rev UNKNOWN.REV
- Verilator: Built from 0.027 MB sources in 2 modules, into 0.008 MB in 4 C++ files needing 0.000 MB
- Verilator: Walltime 0.013 s (elab=0.002, cvt=0.003, bld=0.000); cpu 0.006 s on 1 threads


In [ ]:
import yaml
import subprocess
import tempfile
from pathlib import Path

# === CONFIGURATION ===
yaml_file = "complex_variations_gpt.yml"  # Change to your file path

# === LOAD YAML FILE ===
try:
    with open(yaml_file, "r") as f:
        codes = yaml.safe_load(f)
except FileNotFoundError:
    print(f"❌ YAML file not found: {yaml_file}")
    exit(1)

if not isinstance(codes, list) or not all("code" in item for item in codes):
    print("❌ Invalid YAML format. Expected a list of items with 'code' fields.")
    exit(1)

# === PROMPT FOR INDEX ===
max_index = len(codes) - 1
print(f"📄 Loaded {len(codes)} code snippets from {yaml_file}")

try:
    index = int(input(f"🔢 Enter index to validate (0–{max_index}): "))
except ValueError:
    print("❌ Invalid input. Please enter an integer.")
    exit(1)

if not (0 <= index <= max_index):
    print(f"❌ Invalid index. Must be between 0 and {max_index}.")
    exit(1)

code = codes[index]["code"].strip()

# === WRITE TO TEMP FILE ===
with tempfile.NamedTemporaryFile(delete=False, suffix=".sv", mode="w") as tmp_file:
    tmp_file.write(code)
    tmp_file_path = Path(tmp_file.name)

# === RUN VERILATOR ===
try:
    print(f"\n🎯 Validating index {index} from {yaml_file}")
    result = subprocess.run(
        ["verilator", "--lint-only", "--timing", str(tmp_file_path)],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        timeout=10
    )
    passed = result.returncode == 0
    print("✅ PASS" if passed else "❌ FAIL")

    if result.stdout:
        print("\nSTDOUT:")
        print(result.stdout.decode().strip())
    if result.stderr:
        print("\nSTDERR:")
        print(result.stderr.decode().strip())

except subprocess.TimeoutExpired:
    print("❌ FAIL — Timeout while linting")
except FileNotFoundError:
    print("❌ FAIL — Verilator not found. Is it installed and in PATH?")
finally:
    tmp_file_path.unlink(missing_ok=True)  # Clean up temp file

📄 Loaded 30 code snippets from complex_variations_gpt.yml

🎯 Validating index 0 from complex_variations_gpt.yml
✅ PASS

STDOUT:
- V e r i l a t i o n   R e p o r t: Verilator 5.036 2025-04-27 rev UNKNOWN.REV
- Verilator: Built from 0.028 MB sources in 3 modules, into 0.018 MB in 6 C++ files needing 0.000 MB
- Verilator: Walltime 0.005 s (elab=0.000, cvt=0.003, bld=0.000); cpu 0.005 s on 1 threads


## Validating GPT codes

In [ ]:
import yaml
import subprocess
import tempfile
from pathlib import Path

# === CONFIG ===
input_files = [
    "complex_variations_gpt.yml",
    # "medium@1_variations_gpt.yml",
    # "complex@1_variations_gpt.yml"
]

results = []

for file in input_files:
    with open(file, "r") as f:
        codes = yaml.safe_load(f)

    for i, item in enumerate(codes):
        code = item["code"]
        # Create a temporary .sv file for each code block
        with tempfile.NamedTemporaryFile(delete=False, suffix=".sv", mode="w") as tmp_file:
            tmp_file.write(code)
            tmp_file_path = Path(tmp_file.name)

        # Run verilator lint check
        try:
            result = subprocess.run(
                ["verilator", "--lint-only", "--timing", tmp_file_path],
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                timeout=10
            )
            passed = result.returncode == 0
            results.append({
                "file": file,
                "index": i,
                "passed": passed,
                "stdout": result.stdout.decode(),
                "stderr": result.stderr.decode()
            })
        except subprocess.TimeoutExpired:
            results.append({
                "file": file,
                "index": i,
                "passed": False,
                "stderr": "❌ Timeout while linting",
                "stdout": ""
            })

        # Clean up temp file
        tmp_file_path.unlink()

# === REPORT ===
total = len(results)
passed = sum(1 for r in results if r["passed"])
failed = total - passed

print(f"\n✅ Verilator Validation Summary:")
print(f"Total Files Checked: {total}")
print(f"Passed: {passed}")
print(f"Failed: {failed}\n")

for r in results:
    status = "✅ PASS" if r["passed"] else "❌ FAIL"
    print(f"{status} — {r['file']} [index {r['index']}]")
    if not r["passed"]:
        print(r["stderr"].strip())
        print()


✅ Verilator Validation Summary:
Total Files Checked: 30
Passed: 26
Failed: 4

✅ PASS — complex_variations_gpt.yml [index 0]
✅ PASS — complex_variations_gpt.yml [index 1]
✅ PASS — complex_variations_gpt.yml [index 2]
✅ PASS — complex_variations_gpt.yml [index 3]
✅ PASS — complex_variations_gpt.yml [index 4]
❌ FAIL — complex_variations_gpt.yml [index 5]
%Warning-WIDTHEXPAND: /var/folders/v6/jvlsn67j71g102qybf_hsvxr0000gn/T/tmp69t4l4qm.sv:15:15: Operator INSIDE expects 2 bits on the Inside expression, but Inside expression's VARREF 'w' generates 1 bits.
                                                                                           : ... note: In instance 'mealy'
   15 |         if (w inside {wlist[0], wlist[1]})
      |               ^~~~~~
                      ... For warning description see https://verilator.org/warn/WIDTHEXPAND?v=5.036
                      ... Use "/* verilator lint_off WIDTHEXPAND */" and lint_on around source to disable this message.
%Error-UNSUPPORTED

## Batch inferencing (simple@5, star-coder-3b)

In [ ]:
import yaml
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# === CONFIG ===
code_file = "/tmp/simple_codes.yml"
prompt_file = "/tmp/simple_prompts.yml"
output_file = "/tmp/simple@5_variations.yml"
num_variations_per_prompt = 5

# === LOAD MODEL ===
tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoder2-3b")
model = AutoModelForCausalLM.from_pretrained("bigcode/starcoder2-3b", torch_dtype=torch.float16)
print("✅ Using StarCoder")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# === LOAD INPUT DATA ===
with open(code_file, "r") as f:
    codes = yaml.safe_load(f)

with open(prompt_file, "r") as f:
    prompts = yaml.safe_load(f)

assert len(codes) == len(prompts), "❌ Code and prompt counts do not match!"

# === BUILD PROMPT BATCH ===
batch_prompts = [
    f"### Base Verilog Code:\n{c['code'].strip()}\n### Variation Instruction:\n{p['prompt'].strip()}\n### Modified Code:\n"
    for c, p in zip(codes, prompts)
]

# === RUN MULTIPLE VARIATIONS PER PROMPT ===
outputs = []

for i, prompt in enumerate(batch_prompts):
    responses = generator(
        prompt,
        max_new_tokens=2048,
        do_sample=True,
        temperature=0.8,
        num_return_sequences=num_variations_per_prompt
    )
    for j, resp in enumerate(responses):
        response = resp['generated_text']
        variation = response.split("### Modified Code:\n")[-1].strip()
        outputs.append({"code": variation})
        print(f"✅ [{i+1}/{len(batch_prompts)}] Variation {j+1}/{num_variations_per_prompt} generated")

# === SAVE OUTPUTS WITH BLOCK STYLE ===
def str_presenter(dumper, data):
    if '\n' in data:
        return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
    return dumper.represent_scalar('tag:yaml.org,2002:str', data)

yaml.add_representer(str, str_presenter)

with open(output_file, "w") as f:
    yaml.dump(outputs, f, sort_keys=False)

print(f"\n✅ All {len(outputs)} variations saved to {output_file}")

Device set to use cuda:0


✅ Using StarCoder


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


✅ [1/35] Variation 1/5 generated
✅ [1/35] Variation 2/5 generated
✅ [1/35] Variation 3/5 generated
✅ [1/35] Variation 4/5 generated
✅ [1/35] Variation 5/5 generated


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

✅ [32/35] Variation 1/5 generated
✅ [32/35] Variation 2/5 generated
✅ [32/35] Variation 3/5 generated
✅ [32/35] Variation 4/5 generated
✅ [32/35] Variation 5/5 generated


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


✅ [33/35] Variation 1/5 generated
✅ [33/35] Variation 2/5 generated
✅ [33/35] Variation 3/5 generated
✅ [33/35] Variation 4/5 generated
✅ [33/35] Variation 5/5 generated


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


✅ [34/35] Variation 1/5 generated
✅ [34/35] Variation 2/5 generated
✅ [34/35] Variation 3/5 generated
✅ [34/35] Variation 4/5 generated
✅ [34/35] Variation 5/5 generated
✅ [35/35] Variation 1/5 generated
✅ [35/35] Variation 2/5 generated
✅ [35/35] Variation 3/5 generated
✅ [35/35] Variation 4/5 generated
✅ [35/35] Variation 5/5 generated

✅ All 175 variations saved to /tmp/simple@5_variations.yml


## Batch inferencing (medium@5, star-coder-3b)

In [ ]:
# import yaml
# from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
# import torch

# # === CONFIG ===
# code_file = "/tmp/medium_codes.yml"
# prompt_file = "/tmp/medium_prompts.yml"
# output_file = "/tmp/medium@5_variations.yml"
# num_variations_per_prompt = 5

# # === LOAD MODEL ===
# tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoder2-3b")
# model = AutoModelForCausalLM.from_pretrained("bigcode/starcoder2-3b", torch_dtype=torch.float16)
# print("✅ Using StarCoder")

# generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# # === LOAD INPUT DATA ===
# with open(code_file, "r") as f:
#     codes = yaml.safe_load(f)

# with open(prompt_file, "r") as f:
#     prompts = yaml.safe_load(f)

# assert len(codes) == len(prompts), "❌ Code and prompt counts do not match!"

# # === BUILD AND EXPAND PROMPT BATCH ===
# batch_prompts = []
# source_indices = []

# for i, (c, p) in enumerate(zip(codes, prompts)):
#     prompt_text = f"### Base Verilog Code:\n{c['code'].strip()}\n### Variation Instruction:\n{p['prompt'].strip()}\n### Modified Code:\n"
#     for _ in range(num_variations_per_prompt):
#         batch_prompts.append(prompt_text)
#         source_indices.append(i)

# # === RUN BATCH INFERENCE ===
# responses = generator(
#     batch_prompts,
#     max_new_tokens=2048,
#     do_sample=True,
#     temperature=0.8
# )

# # === EXTRACT VARIATIONS ===
# outputs = []
# counts = [0] * len(codes)

# for i, resp in enumerate(responses):
#     response_text = resp['generated_text']
#     variation = response_text.split("### Modified Code:\n")[-1].strip()
#     outputs.append({"code": variation})
#     src_idx = source_indices[i]
#     counts[src_idx] += 1
#     print(f"✅ [{src_idx+1}/{len(codes)}] Variation {counts[src_idx]}/{num_variations_per_prompt} generated")

# # === SAVE OUTPUTS WITH BLOCK STYLE ===
# def str_presenter(dumper, data):
#     if '\n' in data:
#         return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
#     return dumper.represent_scalar('tag:yaml.org,2002:str', data)

# yaml.add_representer(str, str_presenter)

# with open(output_file, "w") as f:
#     yaml.dump(outputs, f, sort_keys=False)

# print(f"\n✅ All {len(outputs)} variations saved to {output_file}")

In [ ]:
import yaml
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# === CONFIG ===
code_file = "/tmp/medium_codes.yml"
prompt_file = "/tmp/medium_prompts.yml"
output_file = "/tmp/medium@5_variations.yml"
num_variations_per_prompt = 5

# === LOAD MODEL ===
tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoder2-3b")
model = AutoModelForCausalLM.from_pretrained("bigcode/starcoder2-3b", torch_dtype=torch.float16)
print("✅ Using StarCoder")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# === LOAD INPUT DATA ===
with open(code_file, "r") as f:
    codes = yaml.safe_load(f)

with open(prompt_file, "r") as f:
    prompts = yaml.safe_load(f)

assert len(codes) == len(prompts), "❌ Code and prompt counts do not match!"

# === BUILD PROMPT BATCH ===
batch_prompts = [
    f"### Base Verilog Code:\n{c['code'].strip()}\n### Variation Instruction:\n{p['prompt'].strip()}\n### Modified Code:\n"
    for c, p in zip(codes, prompts)
]

# === RUN MULTIPLE VARIATIONS PER PROMPT ===
outputs = []

for i, prompt in enumerate(batch_prompts):
    responses = generator(
        prompt,
        max_new_tokens=2048,
        do_sample=True,
        temperature=0.8,
        num_return_sequences=num_variations_per_prompt
    )
    for j, resp in enumerate(responses):
        response = resp['generated_text']
        variation = response.split("### Modified Code:\n")[-1].strip()
        outputs.append({"code": variation})
        print(f"✅ [{i+1}/{len(batch_prompts)}] Variation {j+1}/{num_variations_per_prompt} generated")

# === SAVE OUTPUTS WITH BLOCK STYLE ===
def str_presenter(dumper, data):
    if '\n' in data:
        return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
    return dumper.represent_scalar('tag:yaml.org,2002:str', data)

yaml.add_representer(str, str_presenter)

with open(output_file, "w") as f:
    yaml.dump(outputs, f, sort_keys=False)

print(f"\n✅ All {len(outputs)} variations saved to {output_file}")

Device set to use cuda:0


✅ Using StarCoder


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


✅ [1/35] Variation 1/5 generated
✅ [1/35] Variation 2/5 generated
✅ [1/35] Variation 3/5 generated
✅ [1/35] Variation 4/5 generated
✅ [1/35] Variation 5/5 generated


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

✅ [32/35] Variation 1/5 generated
✅ [32/35] Variation 2/5 generated
✅ [32/35] Variation 3/5 generated
✅ [32/35] Variation 4/5 generated
✅ [32/35] Variation 5/5 generated


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


✅ [33/35] Variation 1/5 generated
✅ [33/35] Variation 2/5 generated
✅ [33/35] Variation 3/5 generated
✅ [33/35] Variation 4/5 generated
✅ [33/35] Variation 5/5 generated


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


✅ [34/35] Variation 1/5 generated
✅ [34/35] Variation 2/5 generated
✅ [34/35] Variation 3/5 generated
✅ [34/35] Variation 4/5 generated
✅ [34/35] Variation 5/5 generated
✅ [35/35] Variation 1/5 generated
✅ [35/35] Variation 2/5 generated
✅ [35/35] Variation 3/5 generated
✅ [35/35] Variation 4/5 generated
✅ [35/35] Variation 5/5 generated

✅ All 175 variations saved to /tmp/medium@5_variations.yml


In [ ]:
import torch
torch.cuda.empty_cache()

## Batch inferencing (complex@5, star-coder-3b)

In [ ]:
import yaml
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# === CONFIG ===
code_file = "/tmp/complex_codes.yml"
prompt_file = "/tmp/complex_prompts.yml"
output_file = "/tmp/complex@5_variations.yml"
num_variations_per_prompt = 5

# === LOAD MODEL ===
tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoder2-3b")
model = AutoModelForCausalLM.from_pretrained("bigcode/starcoder2-3b", torch_dtype=torch.float16)
print("✅ Using StarCoder")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# === LOAD INPUT DATA ===
with open(code_file, "r") as f:
    codes = yaml.safe_load(f)

with open(prompt_file, "r") as f:
    prompts = yaml.safe_load(f)

assert len(codes) == len(prompts), "❌ Code and prompt counts do not match!"

# === BUILD PROMPT BATCH ===
batch_prompts = [
    f"### Base Verilog Code:\n{c['code'].strip()}\n### Variation Instruction:\n{p['prompt'].strip()}\n### Modified Code:\n"
    for c, p in zip(codes, prompts)
]

# === RUN MULTIPLE VARIATIONS PER PROMPT ===
outputs = []

for i, prompt in enumerate(batch_prompts):
    responses = generator(
        prompt,
        max_new_tokens=2048,
        do_sample=True,
        temperature=0.8,
        num_return_sequences=num_variations_per_prompt
    )
    for j, resp in enumerate(responses):
        response = resp['generated_text']
        variation = response.split("### Modified Code:\n")[-1].strip()
        outputs.append({"code": variation})
        print(f"✅ [{i+1}/{len(batch_prompts)}] Variation {j+1}/{num_variations_per_prompt} generated")

# === SAVE OUTPUTS WITH BLOCK STYLE ===
def str_presenter(dumper, data):
    if '\n' in data:
        return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
    return dumper.represent_scalar('tag:yaml.org,2002:str', data)

yaml.add_representer(str, str_presenter)

with open(output_file, "w") as f:
    yaml.dump(outputs, f, sort_keys=False)

print(f"\n✅ All {len(outputs)} variations saved to {output_file}")

Device set to use cuda:0


✅ Using StarCoder


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


✅ [1/30] Variation 1/5 generated
✅ [1/30] Variation 2/5 generated
✅ [1/30] Variation 3/5 generated
✅ [1/30] Variation 4/5 generated
✅ [1/30] Variation 5/5 generated


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

✅ [27/30] Variation 1/5 generated
✅ [27/30] Variation 2/5 generated
✅ [27/30] Variation 3/5 generated
✅ [27/30] Variation 4/5 generated
✅ [27/30] Variation 5/5 generated


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


✅ [28/30] Variation 1/5 generated
✅ [28/30] Variation 2/5 generated
✅ [28/30] Variation 3/5 generated
✅ [28/30] Variation 4/5 generated
✅ [28/30] Variation 5/5 generated


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


✅ [29/30] Variation 1/5 generated
✅ [29/30] Variation 2/5 generated
✅ [29/30] Variation 3/5 generated
✅ [29/30] Variation 4/5 generated
✅ [29/30] Variation 5/5 generated
✅ [30/30] Variation 1/5 generated
✅ [30/30] Variation 2/5 generated
✅ [30/30] Variation 3/5 generated
✅ [30/30] Variation 4/5 generated
✅ [30/30] Variation 5/5 generated

✅ All 150 variations saved to /tmp/complex@5_variations.yml


## Batch Inferencing (Simple@10, star-coder-3b)

In [ ]:
import yaml
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# === CONFIG ===
code_file = "/tmp/simple_codes.yml"
prompt_file = "/tmp/simple_prompts.yml"
output_file = "/tmp/simple@10_variations.yml"
num_variations_per_prompt = 10

# === LOAD MODEL ===
tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoder2-3b")
model = AutoModelForCausalLM.from_pretrained("bigcode/starcoder2-3b", torch_dtype=torch.float16)
print("✅ Using StarCoder")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# === LOAD INPUT DATA ===
with open(code_file, "r") as f:
    codes = yaml.safe_load(f)

with open(prompt_file, "r") as f:
    prompts = yaml.safe_load(f)

assert len(codes) == len(prompts), "❌ Code and prompt counts do not match!"

# === BUILD PROMPT BATCH ===
batch_prompts = [
    f"### Base Verilog Code:\n{c['code'].strip()}\n### Variation Instruction:\n{p['prompt'].strip()}\n### Modified Code:\n"
    for c, p in zip(codes, prompts)
]

# === RUN MULTIPLE VARIATIONS PER PROMPT ===
outputs = []

for i, prompt in enumerate(batch_prompts):
    responses = generator(
        prompt,
        max_new_tokens=2048,
        do_sample=True,
        temperature=0.8,
        num_return_sequences=num_variations_per_prompt
    )
    for j, resp in enumerate(responses):
        response = resp['generated_text']
        variation = response.split("### Modified Code:\n")[-1].strip()
        outputs.append({"code": variation})
        print(f"✅ [{i+1}/{len(batch_prompts)}] Variation {j+1}/{num_variations_per_prompt} generated")

# === SAVE OUTPUTS WITH BLOCK STYLE ===
def str_presenter(dumper, data):
    if '\n' in data:
        return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
    return dumper.represent_scalar('tag:yaml.org,2002:str', data)

yaml.add_representer(str, str_presenter)

with open(output_file, "w") as f:
    yaml.dump(outputs, f, sort_keys=False)

print(f"\n✅ All {len(outputs)} variations saved to {output_file}")

Device set to use cuda:0


✅ Using StarCoder


FileNotFoundError: [Errno 2] No such file or directory: '/tmp/simple_codes.yml'

## Batch Inferencing (medium@10, star-coder-3b)

In [ ]:
import yaml
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# === CONFIG ===
code_file = "/tmp/medium_codes.yml"
prompt_file = "/tmp/medium_prompts.yml"
output_file = "/tmp/medium@10_variations.yml"
num_variations_per_prompt = 10

# === LOAD MODEL ===
tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoder2-3b")
model = AutoModelForCausalLM.from_pretrained("bigcode/starcoder2-3b", torch_dtype=torch.float16)
print("✅ Using StarCoder")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# === LOAD INPUT DATA ===
with open(code_file, "r") as f:
    codes = yaml.safe_load(f)

with open(prompt_file, "r") as f:
    prompts = yaml.safe_load(f)

assert len(codes) == len(prompts), "❌ Code and prompt counts do not match!"

# === BUILD PROMPT BATCH ===
batch_prompts = [
    f"### Base Verilog Code:\n{c['code'].strip()}\n### Variation Instruction:\n{p['prompt'].strip()}\n### Modified Code:\n"
    for c, p in zip(codes, prompts)
]

# === RUN MULTIPLE VARIATIONS PER PROMPT ===
outputs = []

for i, prompt in enumerate(batch_prompts):
    responses = generator(
        prompt,
        max_new_tokens=2048,
        do_sample=True,
        temperature=0.8,
        num_return_sequences=num_variations_per_prompt
    )
    for j, resp in enumerate(responses):
        response = resp['generated_text']
        variation = response.split("### Modified Code:\n")[-1].strip()
        outputs.append({"code": variation})
        print(f"✅ [{i+1}/{len(batch_prompts)}] Variation {j+1}/{num_variations_per_prompt} generated")

# === SAVE OUTPUTS WITH BLOCK STYLE ===
def str_presenter(dumper, data):
    if '\n' in data:
        return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
    return dumper.represent_scalar('tag:yaml.org,2002:str', data)

yaml.add_representer(str, str_presenter)

with open(output_file, "w") as f:
    yaml.dump(outputs, f, sort_keys=False)

print(f"\n✅ All {len(outputs)} variations saved to {output_file}")

## Batch Inferencing (complex@10, star-coder-3b)

In [ ]:
import yaml
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# === CONFIG ===
code_file = "/tmp/complex_codes.yml"
prompt_file = "/tmp/complex_prompts.yml"
output_file = "/tmp/complex@10_variations.yml"
num_variations_per_prompt = 10

# === LOAD MODEL ===
tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoder2-3b")
model = AutoModelForCausalLM.from_pretrained("bigcode/starcoder2-3b", torch_dtype=torch.float16)
print("✅ Using StarCoder")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# === LOAD INPUT DATA ===
with open(code_file, "r") as f:
    codes = yaml.safe_load(f)

with open(prompt_file, "r") as f:
    prompts = yaml.safe_load(f)

assert len(codes) == len(prompts), "❌ Code and prompt counts do not match!"

# === BUILD PROMPT BATCH ===
batch_prompts = [
    f"### Base Verilog Code:\n{c['code'].strip()}\n### Variation Instruction:\n{p['prompt'].strip()}\n### Modified Code:\n"
    for c, p in zip(codes, prompts)
]

# === RUN MULTIPLE VARIATIONS PER PROMPT ===
outputs = []

for i, prompt in enumerate(batch_prompts):
    responses = generator(
        prompt,
        max_new_tokens=2048,
        do_sample=True,
        temperature=0.8,
        num_return_sequences=num_variations_per_prompt
    )
    for j, resp in enumerate(responses):
        response = resp['generated_text']
        variation = response.split("### Modified Code:\n")[-1].strip()
        outputs.append({"code": variation})
        print(f"✅ [{i+1}/{len(batch_prompts)}] Variation {j+1}/{num_variations_per_prompt} generated")

# === SAVE OUTPUTS WITH BLOCK STYLE ===
def str_presenter(dumper, data):
    if '\n' in data:
        return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
    return dumper.represent_scalar('tag:yaml.org,2002:str', data)

yaml.add_representer(str, str_presenter)

with open(output_file, "w") as f:
    yaml.dump(outputs, f, sort_keys=False)

print(f"\n✅ All {len(outputs)} variations saved to {output_file}")

## Batch Inferencing (RTLCoder-DeepSeek-6.7B, simple@1)

In [ ]:
import yaml
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# === CONFIG ===
code_file = "/tmp/simple_codes.yml"
prompt_file = "/tmp/simple_prompts.yml"
output_file = "/tmp/simple@1_variations.yml"
num_variations_per_prompt = 1

# === LOAD MODEL ===
model_name = "deepseek-ai/deepseek-coder-6.7b-base"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    trust_remote_code=True
).to("cuda")  # manually move to GPU 0

print("✅ Using DeepSeek Coder 6.7B Base")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# === LOAD INPUT DATA ===
with open(code_file, "r") as f:
    codes = yaml.safe_load(f)

with open(prompt_file, "r") as f:
    prompts = yaml.safe_load(f)

assert len(codes) == len(prompts), "❌ Code and prompt counts do not match!"

# === BUILD PROMPT BATCH ===
batch_prompts = [
    f"### Base Verilog Code:\n{c['code'].strip()}\n### Variation Instruction:\n{p['prompt'].strip()}\n### Modified Code:\n"
    for c, p in zip(codes, prompts)
]

# === RUN MULTIPLE VARIATIONS PER PROMPT ===
outputs = []

for i, prompt in enumerate(batch_prompts):
    responses = generator(
        prompt,
        max_new_tokens=2048,
        do_sample=True,
        temperature=0.8,
        num_return_sequences=num_variations_per_prompt
    )
    for j, resp in enumerate(responses):
        response = resp['generated_text']
        variation = response.split("### Modified Code:\n")[-1].strip()
        outputs.append({"code": variation})
        print(f"✅ [{i+1}/{len(batch_prompts)}] Variation {j+1}/{num_variations_per_prompt} generated")

# === SAVE OUTPUTS WITH BLOCK STYLE ===
def str_presenter(dumper, data):
    if '\n' in data:
        return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
    return dumper.represent_scalar('tag:yaml.org,2002:str', data)

yaml.add_representer(str, str_presenter)

with open(output_file, "w") as f:
    yaml.dump(outputs, f, sort_keys=False)

print(f"\n✅ All {len(outputs)} variations saved to {output_file}")

/tmp/conda_envs/hf_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [03:18<00:00, 99.41s/it] 
Device set to use cuda:0


✅ Using DeepSeek Coder 6.7B Base
✅ [1/35] Variation 1/1 generated
✅ [2/35] Variation 1/1 generated
✅ [3/35] Variation 1/1 generated
✅ [4/35] Variation 1/1 generated
✅ [5/35] Variation 1/1 generated
✅ [6/35] Variation 1/1 generated
✅ [7/35] Variation 1/1 generated
✅ [8/35] Variation 1/1 generated
✅ [9/35] Variation 1/1 generated


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


✅ [10/35] Variation 1/1 generated
✅ [11/35] Variation 1/1 generated
✅ [12/35] Variation 1/1 generated
✅ [13/35] Variation 1/1 generated
✅ [14/35] Variation 1/1 generated
✅ [15/35] Variation 1/1 generated
✅ [16/35] Variation 1/1 generated
✅ [17/35] Variation 1/1 generated
✅ [18/35] Variation 1/1 generated
✅ [19/35] Variation 1/1 generated
✅ [20/35] Variation 1/1 generated
✅ [21/35] Variation 1/1 generated
✅ [22/35] Variation 1/1 generated
✅ [23/35] Variation 1/1 generated
✅ [24/35] Variation 1/1 generated
✅ [25/35] Variation 1/1 generated
✅ [26/35] Variation 1/1 generated
✅ [27/35] Variation 1/1 generated
✅ [28/35] Variation 1/1 generated
✅ [29/35] Variation 1/1 generated
✅ [30/35] Variation 1/1 generated
✅ [31/35] Variation 1/1 generated
✅ [32/35] Variation 1/1 generated
✅ [33/35] Variation 1/1 generated
✅ [34/35] Variation 1/1 generated
✅ [35/35] Variation 1/1 generated

✅ All 35 variations saved to /tmp/simple@10_variations.yml


## Batch Inferencing (RTLCoder-DeepSeek-6.7B, medium@1)

In [1]:
import yaml
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# === CONFIG ===
code_file = "/tmp/medium_codes.yml"
prompt_file = "/tmp/medium_prompts.yml"
output_file = "/tmp/medium@1_variations.yml"
num_variations_per_prompt = 1

# === LOAD MODEL ===
model_name = "deepseek-ai/deepseek-coder-6.7b-base"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    trust_remote_code=True
).to("cuda")  # manually move to GPU 0

print("✅ Using DeepSeek Coder 6.7B Base")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# === LOAD INPUT DATA ===
with open(code_file, "r") as f:
    codes = yaml.safe_load(f)

with open(prompt_file, "r") as f:
    prompts = yaml.safe_load(f)

assert len(codes) == len(prompts), "❌ Code and prompt counts do not match!"

# === BUILD PROMPT BATCH ===
batch_prompts = [
    f"### Base Verilog Code:\n{c['code'].strip()}\n### Variation Instruction:\n{p['prompt'].strip()}\n### Modified Code:\n"
    for c, p in zip(codes, prompts)
]

# === RUN MULTIPLE VARIATIONS PER PROMPT ===
outputs = []

for i, prompt in enumerate(batch_prompts):
    responses = generator(
        prompt,
        max_new_tokens=2048,
        do_sample=True,
        temperature=0.8,
        num_return_sequences=num_variations_per_prompt
    )
    for j, resp in enumerate(responses):
        response = resp['generated_text']
        variation = response.split("### Modified Code:\n")[-1].strip()
        outputs.append({"code": variation})
        print(f"✅ [{i+1}/{len(batch_prompts)}] Variation {j+1}/{num_variations_per_prompt} generated")

# === SAVE OUTPUTS WITH BLOCK STYLE ===
def str_presenter(dumper, data):
    if '\n' in data:
        return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
    return dumper.represent_scalar('tag:yaml.org,2002:str', data)

yaml.add_representer(str, str_presenter)

with open(output_file, "w") as f:
    yaml.dump(outputs, f, sort_keys=False)

print(f"\n✅ All {len(outputs)} variations saved to {output_file}")

/tmp/conda_envs/hf_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [02:33<00:00, 76.71s/it] 
Device set to use cuda:0


✅ Using DeepSeek Coder 6.7B Base
✅ [1/35] Variation 1/1 generated
✅ [2/35] Variation 1/1 generated
✅ [3/35] Variation 1/1 generated
✅ [4/35] Variation 1/1 generated
✅ [5/35] Variation 1/1 generated
✅ [6/35] Variation 1/1 generated
✅ [7/35] Variation 1/1 generated
✅ [8/35] Variation 1/1 generated
✅ [9/35] Variation 1/1 generated


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


✅ [10/35] Variation 1/1 generated
✅ [11/35] Variation 1/1 generated
✅ [12/35] Variation 1/1 generated
✅ [13/35] Variation 1/1 generated
✅ [14/35] Variation 1/1 generated
✅ [15/35] Variation 1/1 generated
✅ [16/35] Variation 1/1 generated
✅ [17/35] Variation 1/1 generated
✅ [18/35] Variation 1/1 generated
✅ [19/35] Variation 1/1 generated
✅ [20/35] Variation 1/1 generated
✅ [21/35] Variation 1/1 generated
✅ [22/35] Variation 1/1 generated
✅ [23/35] Variation 1/1 generated
✅ [24/35] Variation 1/1 generated
✅ [25/35] Variation 1/1 generated
✅ [26/35] Variation 1/1 generated
✅ [27/35] Variation 1/1 generated
✅ [28/35] Variation 1/1 generated
✅ [29/35] Variation 1/1 generated
✅ [30/35] Variation 1/1 generated
✅ [31/35] Variation 1/1 generated
✅ [32/35] Variation 1/1 generated
✅ [33/35] Variation 1/1 generated
✅ [34/35] Variation 1/1 generated
✅ [35/35] Variation 1/1 generated

✅ All 35 variations saved to /tmp/medium@1_variations.yml


## Batch Inferencing (RTLCoder-DeepSeek-6.7B, complex@1)

In [1]:
import yaml
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# === CONFIG ===
code_file = "/tmp/complex_codes.yml"
prompt_file = "/tmp/complex_prompts.yml"
output_file = "/tmp/complex@1_variations.yml"
num_variations_per_prompt = 1

# === LOAD MODEL ===
model_name = "deepseek-ai/deepseek-coder-6.7b-base"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    trust_remote_code=True
).to("cuda")  # manually move to GPU 0

print("✅ Using DeepSeek Coder 6.7B Base")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# === LOAD INPUT DATA ===
with open(code_file, "r") as f:
    codes = yaml.safe_load(f)

with open(prompt_file, "r") as f:
    prompts = yaml.safe_load(f)

assert len(codes) == len(prompts), "❌ Code and prompt counts do not match!"

# === BUILD PROMPT BATCH ===
batch_prompts = [
    f"### Base Verilog Code:\n{c['code'].strip()}\n### Variation Instruction:\n{p['prompt'].strip()}\n### Modified Code:\n"
    for c, p in zip(codes, prompts)
]

# === RUN MULTIPLE VARIATIONS PER PROMPT ===
outputs = []

for i, prompt in enumerate(batch_prompts):
    responses = generator(
        prompt,
        max_new_tokens=2048,
        do_sample=True,
        temperature=0.8,
        num_return_sequences=num_variations_per_prompt
    )
    for j, resp in enumerate(responses):
        response = resp['generated_text']
        variation = response.split("### Modified Code:\n")[-1].strip()
        outputs.append({"code": variation})
        print(f"✅ [{i+1}/{len(batch_prompts)}] Variation {j+1}/{num_variations_per_prompt} generated")

# === SAVE OUTPUTS WITH BLOCK STYLE ===
def str_presenter(dumper, data):
    if '\n' in data:
        return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
    return dumper.represent_scalar('tag:yaml.org,2002:str', data)

yaml.add_representer(str, str_presenter)

with open(output_file, "w") as f:
    yaml.dump(outputs, f, sort_keys=False)

print(f"\n✅ All {len(outputs)} variations saved to {output_file}")

/tmp/conda_envs/hf_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [01:37<00:00, 48.77s/it]
Device set to use cuda:0


✅ Using DeepSeek Coder 6.7B Base
✅ [1/30] Variation 1/1 generated
✅ [2/30] Variation 1/1 generated
✅ [3/30] Variation 1/1 generated
✅ [4/30] Variation 1/1 generated
✅ [5/30] Variation 1/1 generated
✅ [6/30] Variation 1/1 generated
✅ [7/30] Variation 1/1 generated
✅ [8/30] Variation 1/1 generated
✅ [9/30] Variation 1/1 generated


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


✅ [10/30] Variation 1/1 generated
✅ [11/30] Variation 1/1 generated
✅ [12/30] Variation 1/1 generated
✅ [13/30] Variation 1/1 generated
✅ [14/30] Variation 1/1 generated
✅ [15/30] Variation 1/1 generated
✅ [16/30] Variation 1/1 generated
✅ [17/30] Variation 1/1 generated
✅ [18/30] Variation 1/1 generated
✅ [19/30] Variation 1/1 generated
✅ [20/30] Variation 1/1 generated
✅ [21/30] Variation 1/1 generated
✅ [22/30] Variation 1/1 generated
✅ [23/30] Variation 1/1 generated
✅ [24/30] Variation 1/1 generated
✅ [25/30] Variation 1/1 generated
✅ [26/30] Variation 1/1 generated
✅ [27/30] Variation 1/1 generated
✅ [28/30] Variation 1/1 generated
✅ [29/30] Variation 1/1 generated
✅ [30/30] Variation 1/1 generated

✅ All 30 variations saved to /tmp/complex@1_variations.yml


In [1]:
import sys

## Batch Inferencing (RTLCoder-DeepSeek-6.7B, simple@5)

In [ ]:
import yaml
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# === CONFIG ===
code_file = "/tmp/simple_codes.yml"
prompt_file = "/tmp/simple_prompts.yml"
output_file = "/tmp/simple@5_variations.yml"
num_variations_per_prompt = 5

# === LOAD MODEL ===
model_name = "deepseek-ai/deepseek-coder-6.7b-base"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    trust_remote_code=True
).to("cuda")  # manually move to GPU 0

print("✅ Using DeepSeek Coder 6.7B Base")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# === LOAD INPUT DATA ===
with open(code_file, "r") as f:
    codes = yaml.safe_load(f)

with open(prompt_file, "r") as f:
    prompts = yaml.safe_load(f)

assert len(codes) == len(prompts), "❌ Code and prompt counts do not match!"

# === BUILD PROMPT BATCH ===
batch_prompts = [
    f"### Base Verilog Code:\n{c['code'].strip()}\n### Variation Instruction:\n{p['prompt'].strip()}\n### Modified Code:\n"
    for c, p in zip(codes, prompts)
]

# === RUN MULTIPLE VARIATIONS PER PROMPT ===
outputs = []

for i, prompt in enumerate(batch_prompts):
    responses = generator(
        prompt,
        max_new_tokens=2048,
        do_sample=True,
        temperature=0.8,
        num_return_sequences=num_variations_per_prompt
    )
    for j, resp in enumerate(responses):
        response = resp['generated_text']
        variation = response.split("### Modified Code:\n")[-1].strip()
        outputs.append({"code": variation})
        print(f"✅ [{i+1}/{len(batch_prompts)}] Variation {j+1}/{num_variations_per_prompt} generated")

# === SAVE OUTPUTS WITH BLOCK STYLE ===
def str_presenter(dumper, data):
    if '\n' in data:
        return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
    return dumper.represent_scalar('tag:yaml.org,2002:str', data)

yaml.add_representer(str, str_presenter)

with open(output_file, "w") as f:
    yaml.dump(outputs, f, sort_keys=False)

print(f"\n✅ All {len(outputs)} variations saved to {output_file}")

sys.exit(0)

/tmp/conda_envs/hf_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [02:33<00:00, 76.59s/it] 
Device set to use cuda:0


✅ Using DeepSeek Coder 6.7B Base
✅ [1/35] Variation 1/5 generated
✅ [1/35] Variation 2/5 generated
✅ [1/35] Variation 3/5 generated
✅ [1/35] Variation 4/5 generated
✅ [1/35] Variation 5/5 generated
✅ [2/35] Variation 1/5 generated
✅ [2/35] Variation 2/5 generated
✅ [2/35] Variation 3/5 generated
✅ [2/35] Variation 4/5 generated
✅ [2/35] Variation 5/5 generated
✅ [3/35] Variation 1/5 generated
✅ [3/35] Variation 2/5 generated
✅ [3/35] Variation 3/5 generated
✅ [3/35] Variation 4/5 generated
✅ [3/35] Variation 5/5 generated
✅ [4/35] Variation 1/5 generated
✅ [4/35] Variation 2/5 generated
✅ [4/35] Variation 3/5 generated
✅ [4/35] Variation 4/5 generated
✅ [4/35] Variation 5/5 generated
✅ [5/35] Variation 1/5 generated
✅ [5/35] Variation 2/5 generated
✅ [5/35] Variation 3/5 generated
✅ [5/35] Variation 4/5 generated
✅ [5/35] Variation 5/5 generated
✅ [6/35] Variation 1/5 generated
✅ [6/35] Variation 2/5 generated
✅ [6/35] Variation 3/5 generated
✅ [6/35] Variation 4/5 generated
✅ [6/35] V

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


✅ [10/35] Variation 1/5 generated
✅ [10/35] Variation 2/5 generated
✅ [10/35] Variation 3/5 generated
✅ [10/35] Variation 4/5 generated
✅ [10/35] Variation 5/5 generated
✅ [11/35] Variation 1/5 generated
✅ [11/35] Variation 2/5 generated
✅ [11/35] Variation 3/5 generated
✅ [11/35] Variation 4/5 generated
✅ [11/35] Variation 5/5 generated
✅ [12/35] Variation 1/5 generated
✅ [12/35] Variation 2/5 generated
✅ [12/35] Variation 3/5 generated
✅ [12/35] Variation 4/5 generated
✅ [12/35] Variation 5/5 generated
✅ [13/35] Variation 1/5 generated
✅ [13/35] Variation 2/5 generated
✅ [13/35] Variation 3/5 generated
✅ [13/35] Variation 4/5 generated
✅ [13/35] Variation 5/5 generated
✅ [14/35] Variation 1/5 generated
✅ [14/35] Variation 2/5 generated
✅ [14/35] Variation 3/5 generated
✅ [14/35] Variation 4/5 generated
✅ [14/35] Variation 5/5 generated
✅ [15/35] Variation 1/5 generated
✅ [15/35] Variation 2/5 generated
✅ [15/35] Variation 3/5 generated
✅ [15/35] Variation 4/5 generated
✅ [15/35] Vari

SystemExit: 0

/tmp/conda_envs/hf_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


: 

## Batch Inferencing (RTLCoder-DeepSeek-6.7B, medium@5)

In [ ]:
import yaml
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# === CONFIG ===
code_file = "/tmp/medium_codes.yml"
prompt_file = "/tmp/medium_prompts.yml"
output_file = "/tmp/medium@5_variations.yml"
num_variations_per_prompt = 5

# === LOAD MODEL ===
model_name = "deepseek-ai/deepseek-coder-6.7b-base"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    trust_remote_code=True
).to("cuda")  # manually move to GPU 0

print("✅ Using DeepSeek Coder 6.7B Base")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# === LOAD INPUT DATA ===
with open(code_file, "r") as f:
    codes = yaml.safe_load(f)

with open(prompt_file, "r") as f:
    prompts = yaml.safe_load(f)

assert len(codes) == len(prompts), "❌ Code and prompt counts do not match!"

# === BUILD PROMPT BATCH ===
batch_prompts = [
    f"### Base Verilog Code:\n{c['code'].strip()}\n### Variation Instruction:\n{p['prompt'].strip()}\n### Modified Code:\n"
    for c, p in zip(codes, prompts)
]

# === RUN MULTIPLE VARIATIONS PER PROMPT ===
outputs = []

for i, prompt in enumerate(batch_prompts):
    responses = generator(
        prompt,
        max_new_tokens=2048,
        do_sample=True,
        temperature=0.8,
        num_return_sequences=num_variations_per_prompt
    )
    for j, resp in enumerate(responses):
        response = resp['generated_text']
        variation = response.split("### Modified Code:\n")[-1].strip()
        outputs.append({"code": variation})
        print(f"✅ [{i+1}/{len(batch_prompts)}] Variation {j+1}/{num_variations_per_prompt} generated")

# === SAVE OUTPUTS WITH BLOCK STYLE ===
def str_presenter(dumper, data):
    if '\n' in data:
        return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
    return dumper.represent_scalar('tag:yaml.org,2002:str', data)

yaml.add_representer(str, str_presenter)

with open(output_file, "w") as f:
    yaml.dump(outputs, f, sort_keys=False)

print(f"\n✅ All {len(outputs)} variations saved to {output_file}")

exit(0)

/tmp/conda_envs/hf_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [06:10<00:00, 185.25s/it]
Device set to use cuda:0


✅ Using DeepSeek Coder 6.7B Base
✅ [1/35] Variation 1/5 generated
✅ [1/35] Variation 2/5 generated
✅ [1/35] Variation 3/5 generated
✅ [1/35] Variation 4/5 generated
✅ [1/35] Variation 5/5 generated
✅ [2/35] Variation 1/5 generated
✅ [2/35] Variation 2/5 generated
✅ [2/35] Variation 3/5 generated
✅ [2/35] Variation 4/5 generated
✅ [2/35] Variation 5/5 generated
✅ [3/35] Variation 1/5 generated
✅ [3/35] Variation 2/5 generated
✅ [3/35] Variation 3/5 generated
✅ [3/35] Variation 4/5 generated
✅ [3/35] Variation 5/5 generated
✅ [4/35] Variation 1/5 generated
✅ [4/35] Variation 2/5 generated
✅ [4/35] Variation 3/5 generated
✅ [4/35] Variation 4/5 generated
✅ [4/35] Variation 5/5 generated
✅ [5/35] Variation 1/5 generated
✅ [5/35] Variation 2/5 generated
✅ [5/35] Variation 3/5 generated
✅ [5/35] Variation 4/5 generated
✅ [5/35] Variation 5/5 generated
✅ [6/35] Variation 1/5 generated
✅ [6/35] Variation 2/5 generated
✅ [6/35] Variation 3/5 generated
✅ [6/35] Variation 4/5 generated
✅ [6/35] V

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


✅ [10/35] Variation 1/5 generated
✅ [10/35] Variation 2/5 generated
✅ [10/35] Variation 3/5 generated
✅ [10/35] Variation 4/5 generated
✅ [10/35] Variation 5/5 generated
✅ [11/35] Variation 1/5 generated
✅ [11/35] Variation 2/5 generated
✅ [11/35] Variation 3/5 generated
✅ [11/35] Variation 4/5 generated
✅ [11/35] Variation 5/5 generated
✅ [12/35] Variation 1/5 generated
✅ [12/35] Variation 2/5 generated
✅ [12/35] Variation 3/5 generated
✅ [12/35] Variation 4/5 generated
✅ [12/35] Variation 5/5 generated
✅ [13/35] Variation 1/5 generated
✅ [13/35] Variation 2/5 generated
✅ [13/35] Variation 3/5 generated
✅ [13/35] Variation 4/5 generated
✅ [13/35] Variation 5/5 generated
✅ [14/35] Variation 1/5 generated
✅ [14/35] Variation 2/5 generated
✅ [14/35] Variation 3/5 generated
✅ [14/35] Variation 4/5 generated
✅ [14/35] Variation 5/5 generated
✅ [15/35] Variation 1/5 generated
✅ [15/35] Variation 2/5 generated
✅ [15/35] Variation 3/5 generated
✅ [15/35] Variation 4/5 generated
✅ [15/35] Vari

: 

## Batch Inferencing (RTLCoder-DeepSeek-6.7B, complex@5)

In [ ]:
import yaml
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# === CONFIG ===
code_file = "/tmp/complex_codes.yml"
prompt_file = "/tmp/complex_prompts.yml"
output_file = "/tmp/complex@5_variations.yml"
num_variations_per_prompt = 5

# === LOAD MODEL ===
model_name = "deepseek-ai/deepseek-coder-6.7b-base"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    trust_remote_code=True
).to("cuda")  # manually move to GPU 0

print("✅ Using DeepSeek Coder 6.7B Base")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# === LOAD INPUT DATA ===
with open(code_file, "r") as f:
    codes = yaml.safe_load(f)

with open(prompt_file, "r") as f:
    prompts = yaml.safe_load(f)

assert len(codes) == len(prompts), "❌ Code and prompt counts do not match!"

# === BUILD PROMPT BATCH ===
batch_prompts = [
    f"### Base Verilog Code:\n{c['code'].strip()}\n### Variation Instruction:\n{p['prompt'].strip()}\n### Modified Code:\n"
    for c, p in zip(codes, prompts)
]

# === RUN MULTIPLE VARIATIONS PER PROMPT ===
outputs = []

for i, prompt in enumerate(batch_prompts):
    responses = generator(
        prompt,
        max_new_tokens=2048,
        do_sample=True,
        temperature=0.8,
        num_return_sequences=num_variations_per_prompt
    )
    for j, resp in enumerate(responses):
        response = resp['generated_text']
        variation = response.split("### Modified Code:\n")[-1].strip()
        outputs.append({"code": variation})
        print(f"✅ [{i+1}/{len(batch_prompts)}] Variation {j+1}/{num_variations_per_prompt} generated")

# === SAVE OUTPUTS WITH BLOCK STYLE ===
def str_presenter(dumper, data):
    if '\n' in data:
        return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
    return dumper.represent_scalar('tag:yaml.org,2002:str', data)

yaml.add_representer(str, str_presenter)

with open(output_file, "w") as f:
    yaml.dump(outputs, f, sort_keys=False)

print(f"\n✅ All {len(outputs)} variations saved to {output_file}")

In [2]:
import torch

torch.cuda.empty_cache()

## Batch Inferencing (RTLCoder-DeepSeek-6.7B, simple@10)

In [3]:
import yaml
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# === CONFIG ===
code_file = "/tmp/simple_codes.yml"
prompt_file = "/tmp/simple_prompts.yml"
output_file = "/tmp/simple@10_variations.yml"
num_variations_per_prompt = 10

# === LOAD MODEL ===
model_name = "deepseek-ai/deepseek-coder-6.7b-base"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    trust_remote_code=True
).to("cuda")  # manually move to GPU 0

print("✅ Using DeepSeek Coder 6.7B Base")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# === LOAD INPUT DATA ===
with open(code_file, "r") as f:
    codes = yaml.safe_load(f)

with open(prompt_file, "r") as f:
    prompts = yaml.safe_load(f)

assert len(codes) == len(prompts), "❌ Code and prompt counts do not match!"

# === BUILD PROMPT BATCH ===
batch_prompts = [
    f"### Base Verilog Code:\n{c['code'].strip()}\n### Variation Instruction:\n{p['prompt'].strip()}\n### Modified Code:\n"
    for c, p in zip(codes, prompts)
]

# === RUN MULTIPLE VARIATIONS PER PROMPT ===
outputs = []

for i, prompt in enumerate(batch_prompts):
    responses = generator(
        prompt,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.8,
        num_return_sequences=num_variations_per_prompt
    )
    for j, resp in enumerate(responses):
        response = resp['generated_text']
        variation = response.split("### Modified Code:\n")[-1].strip()
        outputs.append({"code": variation})
        print(f"✅ [{i+1}/{len(batch_prompts)}] Variation {j+1}/{num_variations_per_prompt} generated")

# === SAVE OUTPUTS WITH BLOCK STYLE ===
def str_presenter(dumper, data):
    if '\n' in data:
        return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
    return dumper.represent_scalar('tag:yaml.org,2002:str', data)

yaml.add_representer(str, str_presenter)

with open(output_file, "w") as f:
    yaml.dump(outputs, f, sort_keys=False)

print(f"\n✅ All {len(outputs)} variations saved to {output_file}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


✅ Using DeepSeek Coder 6.7B Base
✅ [1/35] Variation 1/10 generated
✅ [1/35] Variation 2/10 generated
✅ [1/35] Variation 3/10 generated
✅ [1/35] Variation 4/10 generated
✅ [1/35] Variation 5/10 generated
✅ [1/35] Variation 6/10 generated
✅ [1/35] Variation 7/10 generated
✅ [1/35] Variation 8/10 generated
✅ [1/35] Variation 9/10 generated
✅ [1/35] Variation 10/10 generated
✅ [2/35] Variation 1/10 generated
✅ [2/35] Variation 2/10 generated
✅ [2/35] Variation 3/10 generated
✅ [2/35] Variation 4/10 generated
✅ [2/35] Variation 5/10 generated
✅ [2/35] Variation 6/10 generated
✅ [2/35] Variation 7/10 generated
✅ [2/35] Variation 8/10 generated
✅ [2/35] Variation 9/10 generated
✅ [2/35] Variation 10/10 generated
✅ [3/35] Variation 1/10 generated
✅ [3/35] Variation 2/10 generated
✅ [3/35] Variation 3/10 generated
✅ [3/35] Variation 4/10 generated
✅ [3/35] Variation 5/10 generated
✅ [3/35] Variation 6/10 generated
✅ [3/35] Variation 7/10 generated
✅ [3/35] Variation 8/10 generated
✅ [3/35] Vari

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


✅ [10/35] Variation 1/10 generated
✅ [10/35] Variation 2/10 generated
✅ [10/35] Variation 3/10 generated
✅ [10/35] Variation 4/10 generated
✅ [10/35] Variation 5/10 generated
✅ [10/35] Variation 6/10 generated
✅ [10/35] Variation 7/10 generated
✅ [10/35] Variation 8/10 generated
✅ [10/35] Variation 9/10 generated
✅ [10/35] Variation 10/10 generated
✅ [11/35] Variation 1/10 generated
✅ [11/35] Variation 2/10 generated
✅ [11/35] Variation 3/10 generated
✅ [11/35] Variation 4/10 generated
✅ [11/35] Variation 5/10 generated
✅ [11/35] Variation 6/10 generated
✅ [11/35] Variation 7/10 generated
✅ [11/35] Variation 8/10 generated
✅ [11/35] Variation 9/10 generated
✅ [11/35] Variation 10/10 generated
✅ [12/35] Variation 1/10 generated
✅ [12/35] Variation 2/10 generated
✅ [12/35] Variation 3/10 generated
✅ [12/35] Variation 4/10 generated
✅ [12/35] Variation 5/10 generated
✅ [12/35] Variation 6/10 generated
✅ [12/35] Variation 7/10 generated
✅ [12/35] Variation 8/10 generated
✅ [12/35] Variatio

## Batch Inferencing (RTLCoder-DeepSeek-6.7B, medium@10)

In [ ]:
import yaml
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# === CONFIG ===
code_file = "/tmp/medium_codes.yml"
prompt_file = "/tmp/medium_prompts.yml"
output_file = "/tmp/medium@10_variations.yml"
num_variations_per_prompt = 10

# === LOAD MODEL ===
model_name = "deepseek-ai/deepseek-coder-6.7b-base"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    trust_remote_code=True
).to("cuda")  # manually move to GPU 0

print("✅ Using DeepSeek Coder 6.7B Base")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# === LOAD INPUT DATA ===
with open(code_file, "r") as f:
    codes = yaml.safe_load(f)

with open(prompt_file, "r") as f:
    prompts = yaml.safe_load(f)

assert len(codes) == len(prompts), "❌ Code and prompt counts do not match!"

# === BUILD PROMPT BATCH ===
batch_prompts = [
    f"### Base Verilog Code:\n{c['code'].strip()}\n### Variation Instruction:\n{p['prompt'].strip()}\n### Modified Code:\n"
    for c, p in zip(codes, prompts)
]

# === RUN MULTIPLE VARIATIONS PER PROMPT ===
outputs = []

for i, prompt in enumerate(batch_prompts):
    responses = generator(
        prompt,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.8,
        num_return_sequences=num_variations_per_prompt
    )
    for j, resp in enumerate(responses):
        response = resp['generated_text']
        variation = response.split("### Modified Code:\n")[-1].strip()
        outputs.append({"code": variation})
        print(f"✅ [{i+1}/{len(batch_prompts)}] Variation {j+1}/{num_variations_per_prompt} generated")

# === SAVE OUTPUTS WITH BLOCK STYLE ===
def str_presenter(dumper, data):
    if '\n' in data:
        return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
    return dumper.represent_scalar('tag:yaml.org,2002:str', data)

yaml.add_representer(str, str_presenter)

with open(output_file, "w") as f:
    yaml.dump(outputs, f, sort_keys=False)

print(f"\n✅ All {len(outputs)} variations saved to {output_file}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


✅ Using DeepSeek Coder 6.7B Base
✅ [1/35] Variation 1/10 generated
✅ [1/35] Variation 2/10 generated
✅ [1/35] Variation 3/10 generated
✅ [1/35] Variation 4/10 generated
✅ [1/35] Variation 5/10 generated
✅ [1/35] Variation 6/10 generated
✅ [1/35] Variation 7/10 generated
✅ [1/35] Variation 8/10 generated
✅ [1/35] Variation 9/10 generated
✅ [1/35] Variation 10/10 generated
✅ [2/35] Variation 1/10 generated
✅ [2/35] Variation 2/10 generated
✅ [2/35] Variation 3/10 generated
✅ [2/35] Variation 4/10 generated
✅ [2/35] Variation 5/10 generated
✅ [2/35] Variation 6/10 generated
✅ [2/35] Variation 7/10 generated
✅ [2/35] Variation 8/10 generated
✅ [2/35] Variation 9/10 generated
✅ [2/35] Variation 10/10 generated
✅ [3/35] Variation 1/10 generated
✅ [3/35] Variation 2/10 generated
✅ [3/35] Variation 3/10 generated
✅ [3/35] Variation 4/10 generated
✅ [3/35] Variation 5/10 generated
✅ [3/35] Variation 6/10 generated
✅ [3/35] Variation 7/10 generated
✅ [3/35] Variation 8/10 generated
✅ [3/35] Vari

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


✅ [10/35] Variation 1/10 generated
✅ [10/35] Variation 2/10 generated
✅ [10/35] Variation 3/10 generated
✅ [10/35] Variation 4/10 generated
✅ [10/35] Variation 5/10 generated
✅ [10/35] Variation 6/10 generated
✅ [10/35] Variation 7/10 generated
✅ [10/35] Variation 8/10 generated
✅ [10/35] Variation 9/10 generated
✅ [10/35] Variation 10/10 generated
✅ [11/35] Variation 1/10 generated
✅ [11/35] Variation 2/10 generated
✅ [11/35] Variation 3/10 generated
✅ [11/35] Variation 4/10 generated
✅ [11/35] Variation 5/10 generated
✅ [11/35] Variation 6/10 generated
✅ [11/35] Variation 7/10 generated
✅ [11/35] Variation 8/10 generated
✅ [11/35] Variation 9/10 generated
✅ [11/35] Variation 10/10 generated
✅ [12/35] Variation 1/10 generated
✅ [12/35] Variation 2/10 generated
✅ [12/35] Variation 3/10 generated
✅ [12/35] Variation 4/10 generated
✅ [12/35] Variation 5/10 generated
✅ [12/35] Variation 6/10 generated
✅ [12/35] Variation 7/10 generated
✅ [12/35] Variation 8/10 generated
✅ [12/35] Variatio

: 

## Batch Inferencing (RTLCoder-DeepSeek-6.7B, complex@10)

In [ ]:
import yaml
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# === CONFIG ===
code_file = "/tmp/complex_codes.yml"
prompt_file = "/tmp/complex_prompts.yml"
output_file = "/tmp/complex@10_variations.yml"
num_variations_per_prompt = 10

# === LOAD MODEL ===
model_name = "deepseek-ai/deepseek-coder-6.7b-base"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    trust_remote_code=True
).to("cuda")  # manually move to GPU 0

print("✅ Using DeepSeek Coder 6.7B Base")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# === LOAD INPUT DATA ===
with open(code_file, "r") as f:
    codes = yaml.safe_load(f)

with open(prompt_file, "r") as f:
    prompts = yaml.safe_load(f)

assert len(codes) == len(prompts), "❌ Code and prompt counts do not match!"

# === BUILD PROMPT BATCH ===
batch_prompts = [
    f"### Base Verilog Code:\n{c['code'].strip()}\n### Variation Instruction:\n{p['prompt'].strip()}\n### Modified Code:\n"
    for c, p in zip(codes, prompts)
]

# === RUN MULTIPLE VARIATIONS PER PROMPT ===
outputs = []

for i, prompt in enumerate(batch_prompts):
    responses = generator(
        prompt,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.8,
        num_return_sequences=num_variations_per_prompt
    )
    for j, resp in enumerate(responses):
        response = resp['generated_text']
        variation = response.split("### Modified Code:\n")[-1].strip()
        outputs.append({"code": variation})
        print(f"✅ [{i+1}/{len(batch_prompts)}] Variation {j+1}/{num_variations_per_prompt} generated")

# === SAVE OUTPUTS WITH BLOCK STYLE ===
def str_presenter(dumper, data):
    if '\n' in data:
        return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
    return dumper.represent_scalar('tag:yaml.org,2002:str', data)

yaml.add_representer(str, str_presenter)

with open(output_file, "w") as f:
    yaml.dump(outputs, f, sort_keys=False)

print(f"\n✅ All {len(outputs)} variations saved to {output_file}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


✅ Using DeepSeek Coder 6.7B Base
✅ [1/30] Variation 1/10 generated
✅ [1/30] Variation 2/10 generated
✅ [1/30] Variation 3/10 generated
✅ [1/30] Variation 4/10 generated
✅ [1/30] Variation 5/10 generated
✅ [1/30] Variation 6/10 generated
✅ [1/30] Variation 7/10 generated
✅ [1/30] Variation 8/10 generated
✅ [1/30] Variation 9/10 generated
✅ [1/30] Variation 10/10 generated
✅ [2/30] Variation 1/10 generated
✅ [2/30] Variation 2/10 generated
✅ [2/30] Variation 3/10 generated
✅ [2/30] Variation 4/10 generated
✅ [2/30] Variation 5/10 generated
✅ [2/30] Variation 6/10 generated
✅ [2/30] Variation 7/10 generated
✅ [2/30] Variation 8/10 generated
✅ [2/30] Variation 9/10 generated
✅ [2/30] Variation 10/10 generated
✅ [3/30] Variation 1/10 generated
✅ [3/30] Variation 2/10 generated
✅ [3/30] Variation 3/10 generated
✅ [3/30] Variation 4/10 generated
✅ [3/30] Variation 5/10 generated
✅ [3/30] Variation 6/10 generated
✅ [3/30] Variation 7/10 generated
✅ [3/30] Variation 8/10 generated
✅ [3/30] Vari

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


✅ [10/30] Variation 1/10 generated
✅ [10/30] Variation 2/10 generated
✅ [10/30] Variation 3/10 generated
✅ [10/30] Variation 4/10 generated
✅ [10/30] Variation 5/10 generated
✅ [10/30] Variation 6/10 generated
✅ [10/30] Variation 7/10 generated
✅ [10/30] Variation 8/10 generated
✅ [10/30] Variation 9/10 generated
✅ [10/30] Variation 10/10 generated
✅ [11/30] Variation 1/10 generated
✅ [11/30] Variation 2/10 generated
✅ [11/30] Variation 3/10 generated
✅ [11/30] Variation 4/10 generated
✅ [11/30] Variation 5/10 generated
✅ [11/30] Variation 6/10 generated
✅ [11/30] Variation 7/10 generated
✅ [11/30] Variation 8/10 generated
✅ [11/30] Variation 9/10 generated
✅ [11/30] Variation 10/10 generated
✅ [12/30] Variation 1/10 generated
✅ [12/30] Variation 2/10 generated
✅ [12/30] Variation 3/10 generated
✅ [12/30] Variation 4/10 generated
✅ [12/30] Variation 5/10 generated
✅ [12/30] Variation 6/10 generated
✅ [12/30] Variation 7/10 generated
✅ [12/30] Variation 8/10 generated
✅ [12/30] Variatio

: 

## Batch Inferencing (AutoVCoder, simple@1)

In [ ]:
import yaml
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# === CONFIG ===
code_file = "/tmp/simple_codes.yml"
prompt_file = "/tmp/simple_prompts.yml"
output_file = "/tmp/simple@1_variations.yml"
num_variations_per_prompt = 1

# === LOAD MODEL ===
model_name = "Bin12345/AutoCoder_S_6.7B"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    trust_remote_code=True
).to("cuda")

print("✅ Using AutoVCoder Coder 6.7B Base")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# === LOAD INPUT DATA ===
with open(code_file, "r") as f:
    codes = yaml.safe_load(f)

with open(prompt_file, "r") as f:
    prompts = yaml.safe_load(f)

assert len(codes) == len(prompts), "❌ Code and prompt counts do not match!"

# === BUILD PROMPT BATCH ===
batch_prompts = [
    f"### Base Verilog Code:\n{c['code'].strip()}\n### Variation Instruction:\n{p['prompt'].strip()}\n### Modified Code:\n"
    for c, p in zip(codes, prompts)
]

# === RUN MULTIPLE VARIATIONS PER PROMPT ===
outputs = []

for i, prompt in enumerate(batch_prompts):
    responses = generator(
        prompt,
        max_new_tokens=2048,
        do_sample=True,
        temperature=0.8,
        num_return_sequences=num_variations_per_prompt
    )
    for j, resp in enumerate(responses):
        response = resp['generated_text']
        variation = response.split("### Modified Code:\n")[-1].strip()
        outputs.append({"code": variation})
        print(f"✅ [{i+1}/{len(batch_prompts)}] Variation {j+1}/{num_variations_per_prompt} generated")

# === SAVE OUTPUTS WITH BLOCK STYLE ===
def str_presenter(dumper, data):
    if '\n' in data:
        return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
    return dumper.represent_scalar('tag:yaml.org,2002:str', data)

yaml.add_representer(str, str_presenter)

with open(output_file, "w") as f:
    yaml.dump(outputs, f, sort_keys=False)

print(f"\n✅ All {len(outputs)} variations saved to {output_file}")

## Batch Inferencing (AutoVCoder, medium@1)

In [ ]:
import yaml
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# === CONFIG ===
code_file = "/tmp/medium_codes.yml"
prompt_file = "/tmp/medium_prompts.yml"
output_file = "/tmp/medium@1_variations.yml"
num_variations_per_prompt = 1

# === LOAD MODEL ===
model_name = "Bin12345/AutoCoder_S_6.7B"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    trust_remote_code=True
).to("cuda")

print("✅ Using AutoVCoder Coder 6.7B Base")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# === LOAD INPUT DATA ===
with open(code_file, "r") as f:
    codes = yaml.safe_load(f)

with open(prompt_file, "r") as f:
    prompts = yaml.safe_load(f)

assert len(codes) == len(prompts), "❌ Code and prompt counts do not match!"

# === BUILD PROMPT BATCH ===
batch_prompts = [
    f"### Base Verilog Code:\n{c['code'].strip()}\n### Variation Instruction:\n{p['prompt'].strip()}\n### Modified Code:\n"
    for c, p in zip(codes, prompts)
]

# === RUN MULTIPLE VARIATIONS PER PROMPT ===
outputs = []

for i, prompt in enumerate(batch_prompts):
    responses = generator(
        prompt,
        max_new_tokens=2048,
        do_sample=True,
        temperature=0.8,
        num_return_sequences=num_variations_per_prompt
    )
    for j, resp in enumerate(responses):
        response = resp['generated_text']
        variation = response.split("### Modified Code:\n")[-1].strip()
        outputs.append({"code": variation})
        print(f"✅ [{i+1}/{len(batch_prompts)}] Variation {j+1}/{num_variations_per_prompt} generated")

# === SAVE OUTPUTS WITH BLOCK STYLE ===
def str_presenter(dumper, data):
    if '\n' in data:
        return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
    return dumper.represent_scalar('tag:yaml.org,2002:str', data)

yaml.add_representer(str, str_presenter)

with open(output_file, "w") as f:
    yaml.dump(outputs, f, sort_keys=False)

print(f"\n✅ All {len(outputs)} variations saved to {output_file}")

## Batch Inferencing (AutoVCoder, complex@1)

## Batch Inferencing (AutoVCoder, simple@5)

## Batch Inferencing (AutoVCoder, medium@5)

## Batch Inferencing (AutoVCoder, complex@5)

## Batch Inferencing (AutoVCoder, simple@10)

## Batch Inferencing (AutoVCoder, medium@10)

## Batch Inferencing (AutoVCoder, complex@10)